# Finding Games

To find a single game, use the `LeagueGameFinder` class.
While you can call it without any arguments and get ~30,000 games returned (I believe that's the max number of rows that nba.com will send in a response) across the NBA, WNBA, G-League, and international ball, it's a better idea to pass a team ID.

See `nba_api.stats.static.teams` in the [Basics Notebook](Basics.ipynb) for more detail on getting a team ID.

Let's try to find the last time the Celtics played the Raptors in 2017-18.
That will be four steps:
1. Fetch all Celtics games.
2. Select just games from the 2017-18 season (SEASON_ID ending in 2017).
3. Select games where the opponent is the Raptors (MATCHUP contains 'TOR').
4. Order by date and select the last row.

And last, we'll also get the play-by-play data from that game.

### Get All Celtics Games

In [1]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
celtics = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
celtics_id = celtics['id']

In [2]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612738,BOS,Boston Celtics,0022400021,2024-11-19,BOS vs. CLE,W,240,120,...,0.714,9,33,42,32,4,4,13,19,3.0
1,22024,1610612738,BOS,Boston Celtics,0022400230,2024-11-16,BOS vs. TOR,W,265,126,...,0.895,14,28,42,30,2,6,9,20,3.0
2,22024,1610612738,BOS,Boston Celtics,0022400218,2024-11-13,BOS @ BKN,W,241,139,...,0.708,12,33,45,35,7,6,7,17,25.0
3,22024,1610612738,BOS,Boston Celtics,0022400001,2024-11-12,BOS vs. ATL,L,239,116,...,0.815,6,28,34,28,7,5,20,12,-1.0
4,22024,1610612738,BOS,Boston Celtics,0022400202,2024-11-10,BOS @ MIL,W,240,113,...,0.889,8,34,42,27,5,2,12,16,6.0


As you can see above, the season ID is 5 digits.
I believe the last 4 will always be the current season (2018 for the 2018-19 season).
We can do a sanity check and look at how many games the Celtics have played in recent years.

In [3]:
games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2015':]

,GAME_ID
SEASON_ID,
2015,103
2016,115
2017,113
2018,101
2019,102
2020,80
2021,115
2022,111
2023,111


Note that some of these games are preseason and summer league, so these numbers aren't just regular season and playoffs.

### Filter to Games in the 2017-18 Season

In [4]:
# Subset the games to when the last 4 digits of SEASON_ID were 2017.
games_1718 = games[games.SEASON_ID.str[-4:] == '2017']
games_1718.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
645,42017,1610612738,BOS,Boston Celtics,0041700307,2018-05-27,BOS vs. CLE,L,240,79,...,0.737,11,31,42,18,6,0,5,20,-8.0
646,42017,1610612738,BOS,Boston Celtics,0041700306,2018-05-25,BOS @ CLE,L,241,99,...,0.550,5,26,31,25,5,2,13,18,-10.0
647,42017,1610612738,BOS,Boston Celtics,0041700305,2018-05-23,BOS vs. CLE,W,240,96,...,0.913,7,38,45,18,10,6,8,19,13.0
648,42017,1610612738,BOS,Boston Celtics,0041700304,2018-05-21,BOS @ CLE,L,239,102,...,0.767,9,28,37,21,9,3,9,26,-9.0
649,42017,1610612738,BOS,Boston Celtics,0041700303,2018-05-19,BOS @ CLE,L,240,86,...,0.786,6,28,34,16,4,4,15,25,-30.0


### Filter to Games Against the Raptors

In [5]:
# Subset the games to where MATCHUP contains 'TOR'.
raps_games_1718 = games_1718[games_1718.MATCHUP.str.contains('TOR')]
raps_games_1718.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
668,22017,1610612738,BOS,Boston Celtics,0021701171,2018-04-04,BOS @ TOR,L,237,78,...,0.893,8,35,43,10,7,3,16,16,-18.0
670,22017,1610612738,BOS,Boston Celtics,0021701140,2018-03-31,BOS vs. TOR,W,240,110,...,0.852,8,33,41,19,6,3,5,23,11.0
691,22017,1610612738,BOS,Boston Celtics,0021700798,2018-02-06,BOS @ TOR,L,240,91,...,0.750,9,29,38,21,3,3,17,19,-20.0
732,22017,1610612738,BOS,Boston Celtics,0021700188,2017-11-12,BOS vs. TOR,W,239,95,...,0.789,15,31,46,24,9,2,14,18,1.0


### Sort by Game Date and Select the Last Row

In [6]:
last_raps_game = raps_games_1718.sort_values('GAME_DATE').iloc[-1]
last_raps_game

SEASON_ID                     22017
TEAM_ID                  1610612738
TEAM_ABBREVIATION               BOS
TEAM_NAME            Boston Celtics
GAME_ID                  0021701171
GAME_DATE                2018-04-04
MATCHUP                   BOS @ TOR
WL                                L
MIN                             237
PTS                              78
FGM                              25
FGA                              75
FG_PCT                        0.333
FG3M                              3
FG3A                             22
FG3_PCT                       0.136
FTM                              25
FTA                              28
FT_PCT                        0.893
OREB                              8
DREB                             35
REB                              43
AST                              10
STL                               7
BLK                               3
TOV                              16
PF                               16
PLUS_MINUS                  

There it is.

We can see the game was on April 4th, was in Toronto, and ended in an 18-point Raptors victory.
It can be confusing to read this, but the row is all relative to the Celtics (the team we queried).
All the stats (points, rebounds, blocks, plus/minus) are theirs.

If we wanted stats for both teams (a common use case), we'd need to get *both* rows for this game ID.
There are always two, one with stats for each team.
So back to the `LeagueGameFinder`!

In [7]:
game_id = last_raps_game.GAME_ID
game_id

'0021701171'

In [8]:
# Get **all** the games so we can filter to an individual GAME_ID
result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]
# Find the game_id we want
full_game = all_games[all_games.GAME_ID == game_id]
full_game

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS


Two rows, one with the Celtics' stats and one with the Raptors'.
You may want to join these these two rows into one, so you have stats for both teams in the same observation.

Because this is a common use case, I wrote a function for it.
This function will work for larger datasets too (even though we just have one game here);
you can run it on any game DataFrames.

In [9]:
import pandas as pd

def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    
# Combine the game rows into one. By default, the home team will be TEAM_A.
game_df = combine_team_games(full_game)
game_df

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,...,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B


## Play-by-play Data for a Given Game
With a game ID (which we found above), we can easily fetch play-by-play data for that game.

In [10]:
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2(game_id)
pbp = pbp.get_data_frames()[0]
pbp.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0021701171,2,12,0,1,8:11 PM,12:00,None,Start of 1st Period (8:11 PM EST),None,...,None,None,0,0,None,NaN,None,None,None,0
1,0021701171,4,10,0,1,8:11 PM,12:00,Jump Ball Valanciunas vs. Baynes: Tip to Anunoby,None,None,...,Celtics,BOS,4,1628384,OG Anunoby,1.610613e+09,Toronto,Raptors,TOR,1
2,0021701171,7,2,1,1,8:11 PM,11:38,MISS DeRozan 21' Jump Shot,None,None,...,None,None,0,0,None,NaN,None,None,None,1
3,0021701171,8,4,0,1,8:11 PM,11:36,None,None,Tatum REBOUND (Off:0 Def:1),...,None,None,0,0,None,NaN,None,None,None,1
4,0021701171,9,2,86,1,8:12 PM,11:21,None,None,MISS Horford 12' Turnaround Fadeaway Shot,...,None,None,0,0,None,NaN,None,None,None,1
